In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
from config import  gkey
import gmaps
import time
import json

In [3]:
combined_data = pd.read_csv("Combined_loc_CA.csv")
combined_data.head()

,Zipcode,City,State,CountyName,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,average rent,Median Home Value,Number of House Build in year,Avg_HomeValue,Employment Rate,Latitude,Longitude,Temp
0,90001,Florence-Graham,CA,Los Angeles,57942.0,28.0,34323.0,11294.0,18902.0,32.622277,915.0,272200.0,1950.0,2.999500e+05,44.482413,33.97,-118.25,72.77
1,90002,Los Angeles,CA,Los Angeles,51826.0,27.5,32520.0,11212.0,18302.0,35.314321,902.0,248200.0,1951.0,2.946667e+05,40.938911,34.05,-118.24,73.02
2,90003,Los Angeles,CA,Los Angeles,70208.0,28.0,31878.0,10611.0,24556.0,34.976071,958.0,268500.0,1951.0,3.002417e+05,42.456700,34.05,-118.24,73.02
3,90004,Los Angeles,CA,Los Angeles,63095.0,35.5,43180.0,29194.0,12919.0,20.475473,1044.0,838000.0,1946.0,1.168492e+06,57.777954,34.05,-118.24,73.02
4,90005,Los Angeles,CA,Los Angeles,39338.0,33.9,31485.0,20265.0,11520.0,29.284661,943.0,672600.0,1947.0,7.051917e+05,58.897249,34.05,-118.24,73.02


In [4]:
#create list for Walmart locations
lat_list=[]
lng_list=[]
name_list=[]
address_list=[]
counter =0
#covert city location to a list
locations = combined_data[["Latitude", "Longitude"]].astype(float)
locations_list = locations.values

# geocoordinates
for location in locations_list:
    target_search = "Walmart"
    target_radius = 8000
    target_type = "grocery"
    target_coordinates = str(location[0]) +","+str(location[1])
# set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": gkey
    }

# base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
    response = requests.get(base_url, params=params)
# convert response to json
    places_data = response.json()
    counter+=1
    try:
        for x in range(len(places_data["results"])):
            name = places_data["results"][x]["name"]
            lat =places_data["results"][x]["geometry"]["location"]["lat"]
            lng =places_data["results"][x]["geometry"]["location"]["lng"]
            address =places_data["results"][x]["vicinity"]
#add search result into list
            name_list.append(name)
            lat_list.append(lat)
            lng_list.append(lng)
            address_list.append(address)
    except:
        pass

In [8]:
# combine the result in a data frame
print(counter)
walmart_df = pd.DataFrame({"Name": name_list,"Latitude":lat_list,"Longitude":lng_list,"Address":address_list})
print(walmart_df.count())
walmart_df.head(10)

1066
Name         9864
Latitude     9864
Longitude    9864
Address      9864
dtype: int64


,Name,Latitude,Longitude,Address
0,Walmart Supercenter,33.954813,-118.187613,"4651 Firestone Blvd, South Gate"
1,Walmart Supercenter,33.910008,-118.208162,"2100 N Long Beach Blvd, Compton"
2,Walmart Photo Center,33.954702,-118.187379,"4651 Firestone Blvd, South Gate"
3,Walmart Photo Center,33.910168,-118.208371,"2100 N Long Beach Blvd, Compton"
4,Walmart Vision & Glasses,33.954462,-118.187740,"4651 Firestone Blvd, South Gate"
5,Walmart Grocery Pickup,33.909869,-118.207740,Compton
6,Walmart Grocery Pickup,33.954953,-118.187752,South Gate
7,Walmart,33.956942,-118.272099,"150 E 88th St, Los Angeles"
8,Bellaa inc. Wholesale Supply - eBay Amazon Wal...,33.991743,-118.157664,"6003 Bandini Blvd, Los Angeles"
9,Bellaa inc. Wholesale Supply - eBay Amazon Wal...,33.991743,-118.157664,"6003 Bandini Blvd, Los Angeles"


In [9]:
# remove duplicate rows in dataframe
walmart_df=walmart_df.drop_duplicates()
walmart_df=walmart_df.loc[(walmart_df["Name"]=="Walmart")|(walmart_df["Name"]=="Walmart Supercenter")|(walmart_df["Name"]=="Walmart Neighborhood Market"),:]

walmart_df.count()

Name         281
Latitude     281
Longitude    281
Address      281
dtype: int64

In [10]:
#csv
walmart_df.to_csv("Walmart CA Location.csv",index = False)